<a href="https://colab.research.google.com/github/cloud-commander/face-mask-detection/blob/master/Annotate_Entire_Images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<a href="https://blog.cloudcommander.net" target="_parent"><img src="https://raw.githubusercontent.com/cloud-commander/hexoblog/master/cloud.png" alt="Visit my Blog">
</a>
<br> 
# <span style="font-family:Didot; font-size:3em;"> Cloud Commander </span>

# Pascal VOC Annotation Generation
Creates a Pascal VOC XML annotation file for each image in the supplied folders.

It assumes that the images have already been cropped and the entire image is to be specified as the bounding box.



## Connect to Google Drive

In [1]:
from google.colab import drive

drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [2]:
GDRIVE_FOLDER = "/content/drive/My\ Drive/'Machine Learning'/Datasets"
FILE = "RMFD.zip"

In [11]:
%cd {GDRIVE_FOLDER}
!gsutil cp RMFD.zip /content

/content/drive/My Drive/Machine Learning/Datasets
Copying file://RMFD.zip...
-
Operation completed over 1 objects/610.4 MiB.                                    


In [15]:
%cd /content/

!unzip -o RMFD.zip

Streaming output truncated to the last 5000 lines.
  inflating: self-built-masked-face-recognition-dataset/AFDB_face_dataset/zhouhuimin/1_0_zhouhuimin_0009.jpg  
  inflating: self-built-masked-face-recognition-dataset/AFDB_face_dataset/zhouhuimin/1_0_zhouhuimin_0010.jpg  
  inflating: self-built-masked-face-recognition-dataset/AFDB_face_dataset/zhouhuimin/1_0_zhouhuimin_0011.jpg  
  inflating: self-built-masked-face-recognition-dataset/AFDB_face_dataset/zhouhuimin/1_0_zhouhuimin_0012.jpg  
  inflating: self-built-masked-face-recognition-dataset/AFDB_face_dataset/zhouhuimin/1_0_zhouhuimin_0013.jpg  
  inflating: self-built-masked-face-recognition-dataset/AFDB_face_dataset/zhouhuimin/1_0_zhouhuimin_0014.jpg  
  inflating: self-built-masked-face-recognition-dataset/AFDB_face_dataset/zhouhuimin/1_0_zhouhuimin_0015.jpg  
  inflating: self-built-masked-face-recognition-dataset/AFDB_face_dataset/zhouhuimin/1_0_zhouhuimin_0016.jpg  
  inflating: self-built-masked-face-recognition-dataset/AFDB_

In [45]:
#Generates XML Annotations for images that have already been cropped

import os
from lxml import etree as ET
#import cv2
import numpy as np
from PIL import Image, ImageFile

def annotate_entire_image(IMG_INPUT, CLASS):
	file_counter = 0
	img_counter = 0

	for subdir, dirs, files in os.walk(IMG_INPUT):
		for file in files:
			
			file_counter += 1
			img_path=os.path.join(subdir, file)
			img_name=os.path.basename(img_path)
			

			if img_path.lower().endswith(('.png', '.jpg', '.jpeg')):				
				img = Image.open(img_path)
				h,w,bpp = np.shape(img)
					
				subdir_path, subdir_name = os.path.split(subdir)
				root = ET.Element("annotation", verified="yes")
				ET.SubElement(root, "folder").text = CLASS

				ET.SubElement(root, "filename").text = img_name
				#ET.SubElement(root, "path").text = img_path

				source=ET.SubElement(root, "source")
				ET.SubElement(source, "database").text = "Cloud Commander"

				size=ET.SubElement(root, "size")
				ET.SubElement(size, "width").text = str(w)
				ET.SubElement(size, "height").text = str(h)
				ET.SubElement(size, "depth").text = str(bpp)

				ET.SubElement(root, "segmented").text = "0"

				obj=ET.SubElement(root, "object")
				ET.SubElement(obj, "name").text = CLASS
				ET.SubElement(obj, "pose").text = "Frontal"
				ET.SubElement(obj, "truncated").text = "0"
				ET.SubElement(obj, "difficult").text = "0"

				box=ET.SubElement(obj, "bndbox")
				ET.SubElement(box, "xmin").text = str(0)
				ET.SubElement(box, "ymin").text = str(0)
				ET.SubElement(box, "xmax").text = str(w)
				ET.SubElement(box, "ymax").text = str(h)

				tree = ET.ElementTree(root)
				tree.write(os.path.join(subdir, os.path.splitext(img_name)[0] + '.xml'))
				img_counter += 1

			else :
				with open("delete.txt", "a") as myfile:
						myfile.write(img_path+"\n")
	
	print(f"Result: {file_counter} files were processed and {img_counter} images were annotated")


In [46]:
annotate_entire_image("/content/self-built-masked-face-recognition-dataset/AFDB_masked_face_dataset/","MASKED")

Result: 4406 files were processed and 2203 images were annotated


In [16]:
%cd /content/self-built-masked-face-recognition-dataset/AFDB_face_dataset/

/content/self-built-masked-face-recognition-dataset/AFDB_face_dataset
